In [1]:
import argparse
import torch
import json

from transformers import (
    AutoTokenizer,
    BitsAndBytesConfig
)

import intent_trade_functions as itf
from prompter import PromptManager
from validator import validate_function_call_schema

from utils import (
    print_nous_text_art,
    inference_logger,
    get_assistant_message,
    get_chat_template,
    validate_and_extract_tool_calls
)

In [2]:
prompter = PromptManager()

In [3]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Hermes-2-Pro-Mistral-7B", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

if tokenizer.chat_template is None:
    print("No chat template defined, getting chat_template...")
    tokenizer.chat_template = get_chat_template(chat_template)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
chat = [{"role": "user", "content": "Can you check my balance for me, my telegram id is 4327489"}]
tools = itf.get_openai_tools()
prompt = prompter.generate_prompt(chat, tools, num_fewshot=None)

In [8]:
template_prompt = tokenizer.apply_chat_template(
    prompt,
    add_generation_prompt=True,
    tokenize=False
)

In [9]:
from openai import OpenAI

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
completion = client.completions.create(
    model='NousResearch/Hermes-2-Pro-Mistral-7B',
    prompt=template_prompt,
    max_tokens=1500,
    temperature=0.0,
    stream=True
)

for chunk in completion:
    print(chunk.choices[0].text, end='')

2024-04-19:07:49:00,751 INFO     [_client.py:1026] HTTP Request: POST http://localhost:8000/v1/completions "HTTP/1.1 200 OK"


<tool_call>
{"arguments": {"telegram_id": "4327489"}, "name": "get_portfolio"}
</tool_call>
